In [1]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np
import cv2
import glob
import pathlib
import PIL, PIL.Image
import os
import math
import random
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import (Conv2D, MaxPooling2D, Flatten, Activation, Dropout, Dense, TimeDistributed, LSTM, ConvLSTM2D, MaxPooling3D)

In [2]:
base_dir = '/content/drive/MyDrive'
data_dir = os.path.join(base_dir, 'BSc_Project', 'out')
a_dataset = pathlib.Path(os.path.join(data_dir, 'A'))

In [3]:
image_count = len(list(a_dataset.glob('*/*.png')))
print('number of total images:', image_count, '\n\n')

number of total images: 948 




In [4]:
def load_images(image_dims, data_dir):
  dir = os.listdir(data_dir)
  image_directory = [os.path.join(data_dir, image) for image in dir]
  images = []
  for i, image_path in enumerate(image_directory):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=image_dims)
    input_arr = (tf.keras.preprocessing.image.img_to_array(image))/255
    images.append(input_arr)
  return images

# ------------------------------------------------------
def list_segments(lst, segments):
  arr = np.array(lst)
  return np.array_split(np.array(arr), segments)


In [5]:
norm_images = load_images(image_dims=(224, 224), data_dir=os.path.join(a_dataset, 'norm')) 
sch_images = load_images(image_dims=(224, 224), data_dir=os.path.join(a_dataset, 'sch')) 
norm_cases = list_segments(norm_images, (len(norm_images)//12))
sch_cases = list_segments(sch_images, (len(sch_images)//12))

In [6]:
# ----------------------------------------------
def log_data_shapes(sch, norm):
  return {
 'sch': (len(sch), len(sch[0]), sch[0][0].shape), 
 'norm': (len(norm), len(norm[0]), norm[0][0].shape)}

log_data_shapes(sch_cases, norm_cases)

{'sch': (42, 12, (224, 224, 3)), 'norm': (37, 12, (224, 224, 3))}

In [7]:
def attach_labels(dataset, label):
  return [[sample, label] for sample in dataset]

dataset = []
dataset.extend(attach_labels(sch_cases, 0))
dataset.extend(attach_labels(norm_cases, 1))

In [8]:
def log_shape(dataset):
  print((len(dataset), len(dataset[0]), len(dataset[0][0]), (len(dataset[0][0][0]), len(dataset[0][0][0][0]), len(dataset[0][0][0][0][0]))))

In [9]:
def split_train_test(dataset, ratio=0.8, seed=123):
  train_len = math.floor(len(dataset) * ratio)
  random.seed(seed)
  random.shuffle(dataset)
  return dataset[:train_len], dataset[train_len:]

def split_data_label(dataset):
  X = []
  y = []
  for sample, label in dataset:
    X.append(sample)
    y.append(label)
  return np.array(X), np.array(y)


train_data, test_data = split_train_test(dataset)
print(len(train_data), len(test_data))
X_train, y_train = split_data_label(train_data)
X_test, y_test = split_data_label(test_data)

63 16


In [10]:
log_shape(train_data)
log_shape(test_data)
print(len(X_train), len(X_train[0]), (len(X_train[0][0]), len(X_train[0][0][0]), len(X_train[0][0][0][0])))

(63, 2, 12, (224, 224, 3))
(16, 2, 12, (224, 224, 3))
63 12 (224, 224, 3)


In [12]:
models_path = os.path.join(base_dir, 'BSc_Project', 'models')

checkpoint = keras.callbacks.ModelCheckpoint(
    os.path.join(models_path, 'cnn_lstm_model.h5'),
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1)


In [13]:
model = Sequential()
       
model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu'), input_shape = (12, 224, 224, 3)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2)))) 
model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same',activation = 'relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))                
model.add(TimeDistributed(Flatten()))
                                  
model.add(LSTM(64))
model.add(Dense(512, activation = "relu"))               
model.add(Dense(2, activation = "softmax"))

model.summary()

optimizer = keras.optimizers.Adam(learning_rate=0.00009)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer=optimizer, metrics=["accuracy"])

model_history = model.fit(X_train, y_train, epochs=60, batch_size=8 ,validation_split=0.2, callbacks = [checkpoint])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 12, 224, 224, 32)  896      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 12, 112, 112, 32)  0        
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 12, 112, 112, 64)  18496    
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 12, 56, 56, 64)   0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 12, 56, 56, 128)  7

In [14]:
loss, acc = model.evaluate(X_test, y_test, batch_size= 4)
print(f"model's accuracy: {round((acc * 100), 2)}%")

4/4 [==============================] - 1s 42ms/step - loss: 0.5409 - accuracy: 0.8125
model's accuracy: 81.25%


In [23]:
def predict(model):
  return np.argmax(model.predict([X_test]), axis=-1)

print(f'Predictions: {predict(model)},\nTrue label:  {y_test}')

1/1 [==============================] - 0s 196ms/step
Predictions: [0 0 0 0 1 1 1 1 1 0 0 1 1 1 0 0],
True label:  [1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 0]
